In [7]:
# Standard libraries of Python
import re

# Dependencies
import pandas as pd
import requests

from bs4 import BeautifulSoup,Tag
from dataclasses import dataclass
from selectolax.parser import HTMLParser

# Libraries
from backend.database.src.creature import Pokemon, Mega_Pokemon
from backend.database.src import parse

In [ ]:
# Class for selectolax scrap
# @dataclass
# class Pokemon:
#     name: str
#     gender: str
#     types: list

In [ ]:
#Selecto scrap
pokedex = []
url = 'https://www.serebii.net/pokedex-sm/{}.shtml'.format(str(1).zfill(3))
html = requests.get(url).text
tree = HTMLParser(html)
pokemon = Pokemon(name=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(1)').text(),
            gender=tree.css_first('#content > main > div > div > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(4)').text(),
            types=list(map(lambda x: x.attributes['alt'], tree.css('#content > main > div > div > table:nth-child(5) > tbody > tr:nth-child(2) > td.cen >> img'))))
pokedex.append(pokemon)

pd.DataFrame(pokedex)

In [ ]:
#Examble of use with bs4
html_text = requests.get(f'https://www.serebii.net/pokedex-xy/006.shtml').text
soup = BeautifulSoup(html_text, 'html.parser')
all_divs = soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[0].find_all('td', {'class': 'footype'})

In [ ]:
for tag in foo_info:
    a_tag = tag.find('img')
    text = a_tag['src']
    if 'normal' in text:
        print('esta')
    else:
        print('xd')

In [36]:
# Class test
x = Pokemon(8,'raichu')

all_divs = x.soup.find_all('div', attrs={'align': 'center'})
foo_info = all_divs[1].find_all('table', {'class': 'dextable'})

x.name()

In [23]:
parse.elements_atk(foo_info[15].find_all('td')[2])

KeyError: 'src'

In [8]:
ala = parse.elements_atk(foo_info[15].find_all('td')[2],1)

KeyError: 'src'

In [37]:
info = []
for i in foo_info[12].find_all('td'):
    info.append(i)

In [38]:
data = info[1:]
org = [item for sublist in data for item in sublist]
reshape = [
    org[i:i+9]
    for i in range(0,len(org),9)
]

In [40]:
org

[<a href="/itemdex/tm00.shtml">TM00</a>,
 <a href="/attackdex-swsh/megapunch.shtml">Mega Punch</a>,
 <img alt="Mega Punch - Normal-type" loading="lazy" src="/pokedex-bw/type/normal.gif"/>,
 <img alt="Mega Punch: Physical Move" loading="lazy" src="/pokedex-bw/type/physical.png"/>,
 '80',
 '85',
 '20',
 '--',
 <table><tr><td class="pkmn"><img alt="Normal" loading="lazy" src="/pokedex-swsh/icon/026.png"/></td><td class="pkmn"><img alt="Alolan Form" loading="lazy" src="/pokedex-swsh/icon/026-a.png"/></td></tr></table>,
 <img alt="Normal" loading="lazy" src="/pokedex-swsh/icon/026.png"/>,
 <img alt="Alolan Form" loading="lazy" src="/pokedex-swsh/icon/026-a.png"/>,
 'The target is slugged by a punch thrown with muscle-packed power.',
 <a href="/itemdex/tm01.shtml">TM01</a>,
 <a href="/attackdex-swsh/megakick.shtml">Mega Kick</a>,
 <img alt="Mega Kick - Normal-type" loading="lazy" src="/pokedex-bw/type/normal.gif"/>,
 <img alt="Mega Kick: Physical Move" loading="lazy" src="/pokedex-bw/type/ph

In [ ]:
data = info[1:]
to_fill = []

i = 0
while i < len(data):
    l = 11 if i == 0 or isinstance(data[i+9],str) else 10
    to_fill.extend([data[i:i+l]])
    i += l

    print(i)

In [ ]:
to_delete = []

# Suponiendo que `foo_info[12]` contiene la lista de elementos td que proporcionaste
for n, td_element in enumerate(info):
    skip_row = True
    
    try:
        pkmn_elements = td_element.find_all(class_='pkmn')

        for pkmn in pkmn_elements:
            if skip_row:
                e = n

            img_element = pkmn.find('img')
            n += 1

            if img_element:
                alt_text = img_element.get('alt', 'No alt text')
                del info[n]
                info.insert(n,alt_text)
                print(alt_text)
            
            if skip_row:
                to_delete.append(e)
                skip_row = False
                
    except AttributeError:
        pass

In [ ]:
for idx in reversed(to_delete):
    del info[idx]

In [ ]:
data = info[1:]

i = 0
while i < len(data):
    print(data[i+8])
    l = 11 if i == 0 or isinstance(data[i+9],str) else 10
    if l == 10 and any(word in data[i+8] for word in ['Alolan', 'Galarian', 'Hisuian', 'Paldean']):
        fix = data[i:i+l]
        fix.insert(8, 'N/A')
        to_fill.extend(fix)
    else:
        to_fill.extend([data[i:i+l]])
    i += l 

    print(i)

In [ ]:
mask_abilities = {}
terastallised_abilities = {}

for i in foo_info[10].find_all('b'):
    father = i
    ability = father.text
    if ' Mask ' in ability:
        mask_abilities[ability] = []
    elif ' Mask' in ability:
        terastallised_abilities[ability] = []
    else:
        print(ability)

In [ ]:
ala = foo_info[10].find_all('b')

In [ ]:
for n,i in enumerate(ala):
    try:
        father = i
        ability = father.text

        if ' Mask ' in ability:
            continue

        ability_text = father.next_element.next.strip()
        to_join = [ability, ability_text]
        result = ''.join(to_join)
        print(result)

        for e,k in enumerate(mask_abilities):
            if n == e:
                mask_abilities[k] = result
                break

        print(mask_abilities)
    except TypeError:
        print('error')
        continue

In [ ]:
for n,i in enumerate(mask_abilities):
    print(n,i)

In [ ]:
terastallised_abilities

In [ ]:
elemental_types = ['Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fight', 'Poison', 'Ground', 'Flying', 'Psychc', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy']

In [ ]:
for i in elemental_types:
    print(i.capitalize())

In [ ]:
all_divs[1].find('td', string=re.compile("Base Stats - Total.*")).find_next_siblings('td')

In [ ]:
for i in foo_info[0]:
    print(i.split())
    a_tag = i.find('img')
    print(a_tag)
    type_text = a_tag['alt']
    print(type_text)

In [ ]:
type(foo_info[0].find(string='XD'))

In [ ]:
ala = {'regional': 0}

if 'regional' in ala:
    print(ala)

In [ ]:
m = Mega_Pokemon(x)

In [ ]:
m.name()

In [ ]:
m.m_name

In [ ]:
s = all_divs[0].find_all('table', {'class': 'dextable'})

In [ ]:
for i, table in enumerate(s):
    if table.find('td', {'class': 'fooevo', 'colspan': '6'}):
        print('existe')
        print(i)

In [ ]:
#Abilities
ala = s[20].find_all('b')

abilities = {'ability': []}

parse.process_ability(ala[2], abilities, None, None)


In [ ]:
abilities

In [ ]:
#Weakness
types = []

for td_tag in foo_info[0:18]:
    a_tag = td_tag.find('img')
    if a_tag:
        type_text = a_tag['alt']
        types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

print(types)

filtered = [tag for tag in foo_info[18:36] if '*' in tag.get_text(strip=True)]
filtered_t = [i.text for i in filtered]
val = list(map(lambda x: int(x.replace('*','')),filtered_t))

In [ ]:
#Elemental Types
tp = foo_info[0]
types = []
for tag in tp:
    a_tag = tag.find('img')
    print(a_tag)
    type_text = a_tag['alt']
    print(type_text)
    types.append(type_text)

s = 'Attacking Move Type: ','-type'
for string in s:
    types = list(map(lambda x: x.replace(string,''),types))

In [ ]:
base_form = []
form = []

if foo_info[0].find(string='Normal'):
    main = foo_info[0].find_all('td')

    for index,tag in enumerate(main):
        lazy = tag.text.strip()

        if lazy == 'Normal':
            basic = main[index+1].find('img')
            type_text = basic['alt']
            base_form.append(type_text)
        elif lazy:
            basic = main[index+1].find_all('img')
            for e in basic:
                type_text = e['alt']
                form.append(type_text)

In [ ]:
#Get al tables from the main center table
tables = all_divs[0].find_all('table', {'class': 'dextable'})

#Search for the line that contains <td class="fooevo" colspan="6">
mega_evolution_position = -1
counter = 0
for i, table in enumerate(tables):
    if table.find('td', {'class': 'fooevo', 'colspan': '6'}):
        mega_evolution_position = i
        print(i)
        print(table)
        break #Simulates return
    else:
        print('Does not have Mega')

In [ ]:
#Mega Evolution Types
t = tables[19].find('td', class_='cen')

ty_val = []

for i in t:
    n = i.find('img')
    if n and not isinstance(n,Tag):
        continue
    n = n['src']
    for element in x.elemental_types:
        ty = element.lower()
        if ty in n:
            ty_val.append(element)

print(ty_val)

In [ ]:
#Mega Evolution Weakness
w = []

if ty_val == x._elements:
    pass
else:
    weak_location = tables[22].find('td', string=re.compile("Base Stats - Total.*")).find_next_siblings('td')
    filtered_types = list(filter(lambda x: '*' in x.text, weak_location.find_all('td', {'class': 'footype'})))
    filtered = [tag.text for tag in filtered_types if '*' in tag.get_text(strip=True)]

    val = list(map(lambda x: x.replace('*',''),filtered_t))

    for i in val:
        try:
            w.append(int(i))
        except ValueError:
            w.append(float(i))

    mega_weak = dict(zip(x.elemental_types,w))